for the course "<a target="_blank" href="https://www.udemy.com/course/python-3-deep-dive-part-1/">Python 3: Deep Drive (part 1 - Functional)</a>",<br>
section 7: "Scopes, Closures and Decorators",<br>
(Q&A for <a target="_blank" href="https://www.udemy.com/course/python-3-deep-dive-part-1/learn/lecture/9376820#questions/8320248">lecture 116</a>)

<h3>Different scopes</h3>

In [1]:
a = 'global variable a'
b = 'global variable b'
c = 'global variable c'
d = 'global variable d'
e = 'global variable e'

def outer_func():
    a = 'outer_func a'
    b = 'outer_func b'
    c = 'outer_func c'
    d = 'outer_func d'
    # e - not mentioned
    
    def inner_func():
        global a
        nonlocal b
        c = 'inner_func c'
        # d - not mentioned: we can read it
        #     but attempt assign to it create local variable
        # nonlocal e → error
        print(a, b, c, d, sep='\n')

    inner_func()

outer_func()

global variable a
outer_func b
inner_func c
outer_func d


<br>
<br>
<hr>
<h3>Closure</h3>

In [2]:
# Simplest closure
def outer():
    x = 'Python'                # 'x' → free variable
    
    def inner():                # 'inner()' → inner function
        print(f'{x} rocks!')    # 'inner()' + 'x' → closure
    
    return inner                # "returning" a closure

fn = outer()

fn()                            #> Python rocks!
fn()                            #> Python rocks!
print()

print(fn.__closure__)           #> (<cell at 0x…: str object at 0x…>,)
print(fn.__code__.co_freevars)  #> ('x',)

Python rocks!
Python rocks!

(<cell at 0x7fbbb0445768: str object at 0x7fbbb56d22d0>,)
('x',)


↑ <code>outer.x</code> and <code>inner.x</code> point to <b>intermediary cell</b>, that in its turn point to common value.<br>Therefore, if <code>x</code> changes it changes simultaneously for both scopes.<br><br>

In [3]:
# Simplest closure with changeable free variable
def outer():
    x = 'Python'
    n = 0
    
    def inner():
        nonlocal n
        n += 1
        print(f'{x} rocks! -{n}')
    
    return inner

fn1 = outer()
fn2 = outer()

fn1()                            #> Python rocks! -1
fn1()                            #> Python rocks! -2
print()

fn2()                            #> Python rocks! -1
fn2()                            #> Python rocks! -2
print()

print(fn1.__closure__)           #> (<cell at 0x…: int object at 0x…>,
                                 #>  <cell at 0x…: str object at 0x…>)
print(fn1.__code__.co_freevars)  #> ('n', 'x')

Python rocks! -1
Python rocks! -2

Python rocks! -1
Python rocks! -2

(<cell at 0x7fbbb0445678: int object at 0x56367c0cb440>, <cell at 0x7fbbb04456a8: str object at 0x7fbbb56d22d0>)
('n', 'x')


In [4]:
# Closure returning 2 functions with common free variable
def outer():
    count = 0
    
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count
    
    return inc1, inc2

g1, g2 = outer()
print(g1(), g1(), g2(), g1())  #> 1 2 3 4

1 2 3 4


In [6]:
# Closure with settable during run-time initial value of a free variable
def adder(n):
    def inner(x):
        return x+n
    return inner

add_from_10 = adder(10)
add_from_20 = adder(20)

print( add_from_10(5) )  #> 15
print( add_from_20(5) )  #> 25

15
25


In [7]:
# POTENTIAL DANGER with unintentional closure
adders = []

for n in range(10,40,10):
    adders.append(lambda x: x+n)

for adr in adders:
    print(adr(5))
print()
#> 35
#> 35
#> 35

n = 100
for adr in adders:
    print(adr(5))
print()
#> 105
#> 105
#> 105

for adr in adders:
    print(adr.__closure__)
print()
#> None
#> None
#> None

for adr in adders:
    print(adr.__code__.co_freevars)
print()
#> ()
#> ()
#> ()

for adr in adders:
    print(adr)
print()
#> <function <lambda> at 0x7fc2d41f1620>
#> <function <lambda> at 0x7fc2d41f1f28>
#> <function <lambda> at 0x7fc2d411c488>

35
35
35

105
105
105

None
None
None

()
()
()

<function <lambda> at 0x7fbbb03f6048>
<function <lambda> at 0x7fbbb03f60d0>
<function <lambda> at 0x7fbbb03f6158>



In [8]:
# correction of the previous code
adders = []

for n in range(10,40,10):
    adders.append(lambda x, *, start=n: start+x)

for adr in adders:
    print(adr(5))
print()
#> 15
#> 25
#> 35

# but caution for such case
for adr in adders:
    print(adr(5, start=60))
#> 65
#> 65
#> 65

15
25
35

65
65
65


<br>
<hr />
<h4>Nested closure</h4>

In [9]:
# nested closure -1    # skip
def incrementer(n):
    
    # inner + n → outer closure
    def inner(start):
        current = start
        
        # inc + current + n → nested closure
        def inc():
            nonlocal current
            current += n
            return current

        return inc
    
    return inner

inc_2 = incrementer(2)
inc_2_from_100 = inc_2(100)

print(inc_2_from_100())  #> 102
print(inc_2_from_100())  #> 104

102
104


In [10]:
# nested closure -2
def incrementer(start):
    
    # inner + start → outer closure
    def inner(n):
        current = start
        
        # inc + n + start → nested closure
        def inc():
            nonlocal current
            current += n
            return current

        return inc
    
    return inner

inc_from_100 = incrementer(100)
inc_2_from_100 = inc_from_100(2)

print(inc_2_from_100())  #> 102
print(inc_2_from_100())  #> 104

102
104


In [11]:
# example with nested closure
def pow(n):
    def inner(x):
        return x**n
    return inner

square = pow(2)
print(square(3))
print(square(10))
print()

print(square.__closure__)
print(square.__code__.co_freevars)

9
100

(<cell at 0x7fbbb0445348: int object at 0x56367c0cb440>,)
('n',)


---
Application 1 (finding average of received numbers)

In [13]:
def averager():
    total = 0
    count = 0
    
    def add(n):
        nonlocal total, count
        total += n
        count += 1
        return total / count
    
    return add


a = averager()

print(a(10))
print(a(20))
print(a(30))

10.0
15.0
20.0


In [14]:
print(a.__closure__)
print(a.__code__.co_freevars)

(<cell at 0x7fbbb0445d38: int object at 0x56367c0cb460>, <cell at 0x7fbbb0445a08: int object at 0x56367c0cbb80>)
('count', 'total')


---
Application 2 (simplest timer)

In [15]:
from time import perf_counter

def simplest_timer():
    start = perf_counter()
    
    def poll():
        current = perf_counter()
        return round(current - start, 3)
    
    return poll


t = simplest_timer()

In [16]:
t()

3.021

In [17]:
t()

8.675

In [18]:
print(t.__closure__)
print(t.__code__.co_freevars)

(<cell at 0x7fbbb04455b8: float object at 0x7fbbb044f558>,)
('start',)


---
Application 3 (simplest counter)

In [19]:
def counter(c=0):
    def inc(increment=1):
        nonlocal c
        c += increment
        return c
    return inc
    
counter1 = counter(10)
print(counter1())
print(counter1())

11
12


---
Application 4 (counter of function calls)

In [20]:
def counter_func(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'{fn.__name__} has been called {cnt} times')
        return(fn(*args, **kwargs))
    return inner

counter2 = counter_func(print)
counter2('E')
counter2('l')
print()

counter_add = counter_func(lambda a,b: a+b)
result = counter_add(1, 2)
result = counter_add(3, 4)

print has been called 1 times
E
print has been called 2 times
l

<lambda> has been called 1 times
<lambda> has been called 2 times


In [21]:
print(counter_add.__closure__)
print(counter_add.__code__.co_freevars)

(<cell at 0x7fbbb0445df8: int object at 0x56367c0cb440>, <cell at 0x7fbbb0445d98: function object at 0x7fbbb03f6ea0>)
('cnt', 'fn')


In [22]:
# more sophisticated version of the application,
# the results are saved in outer dictionary

counters = dict()

def counter_func_general(fn, counters):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt
        # print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args, **kwargs)
    return inner


def add(a, b):
    return a+b


counter_add = counter_func_general(add, counters)
counter_add(1,2)
counter_add(3,4)

print(counters)

{'add': 2}


In [23]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product


fact = counter_func_general(fact, counters)
fact(0)
fact(6)

print(counters)

{'add': 2, 'fact': 2}


In [24]:
print(fact.__closure__)

(<cell at 0x7fbbb04457f8: int object at 0x56367c0cb440>, <cell at 0x7fbbb04454f8: dict object at 0x7fbbb0407ab0>, <cell at 0x7fbbb04451f8: function object at 0x7fbbb03f6e18>)


In [25]:
print(fact.__code__.co_freevars)

('cnt', 'counters', 'fn')


In [26]:
print(fact.__name__)

inner
